In [58]:
import sys
import os
sys.path.append(os.getcwd())
import pandas as pd
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as stemmer 
import nltk
from nltk.corpus import stopwords
from collections import Counter
import nltk.stem

# Special vocabulary module from shoyu
import vocabulary_hdp as vocab

In [38]:
stemmer = stemmer("english")

In [39]:
def preprocess(doc):
    return [stemmer.stem(WordNetLemmatizer().lemmatize(w, pos='v')) for w in doc.translate(str.maketrans('','', string.punctuation)).lower().split(' ')]

def rm_stopwords_and_short_words(words):
    results = []
    for i in words:
        if not i in stopwords1 and len(i)  > 3:
            results.append(i)
    return results

def full_preprocess(doc):
    return rm_stopwords_and_short_words(preprocess(doc))


In [40]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andrewcarr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrewcarr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
df = pd.read_csv("tm_test_data.csv")

FileNotFoundError: [Errno 2] File b'tm_test_data.csv' does not exist: b'tm_test_data.csv'

In [43]:
stopwords1 = stopwords.words('english')

In [45]:
tokenized_df = [full_preprocess(i) for i in df.abstract]

In [46]:
all_words = [i for sublist in tokenized_df for i in sublist]
all_words =  list(set(all_words))

In [47]:
# Filter out tokens that appear in fewer than 3 abstracts and tokens that appear in more than half the abstracts 
all_words_counts = np.zeros(len(all_words))

for k,i in enumerate(all_words):
    for j in tokenized_df:
        if i in j:
            all_words_counts[k] += 1 
            
word_counts_dict = list(zip(all_words, list(all_words_counts)))
word_counts_dict_ab = list(filter(lambda x: x[1] > 3, word_counts_dict))
word_counts_dict_ab2 = list(filter(lambda x: x[1] < len(tokenized_df)/2, word_counts_dict_ab))
final_dict = [i[0] for i in word_counts_dict_ab2]

In [48]:
tokenized_df_ab = []
for i in tokenized_df:
    tokenized_df_ab.append([j for j in i if j in final_dict])
    

In [52]:
voca = vocab.Vocabulary()
docs = [voca.doc_to_ids(doc) for doc in tokenized_df_ab]
beta = .5

In [61]:
! jupyter nbconvert --to script 'data_prep.ipynb'

[NbConvertApp] Converting notebook data_prep.ipynb to script
[NbConvertApp] Writing 2210 bytes to data_prep.py
